## Method2

In [1]:
import trimesh
import pyvista as pv
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/two_spheres2.ply")

In [3]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1118 |
+-----------------------------------+---------+
| Number of faces                   |    1920 |
+-----------------------------------+---------+
| Number of intersecting face pairs |    2354 |
+-----------------------------------+---------+


#### Get outer hull

In [7]:
mesh = pymesh.compute_outer_hull(mesh)

In [8]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     842 |
+-----------------------------------+---------+
| Number of faces                   |    1680 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


#### Create a bounding box

In [9]:
local_min = np.array([ -0.5, -0.5, -0.7])
local_max = np.array([ 1.05,  1.0,  1.0])

In [10]:
submesh, face_mask = extract_submesh_by_bbox(mesh, local_min, local_max)

In [11]:
visualize(submesh, filename="Submesh")

Widget(value='<iframe src="http://localhost:44475/index.html?ui=P_0x7f03e43e0070_0&reconnect=auto" class="pyvi…

None

In [12]:
remaining_mesh = extract_remaining_mesh(mesh, face_mask)

In [13]:
visualize(remaining_mesh, filename="remaining_mesh")

Widget(value='<iframe src="http://localhost:44475/index.html?ui=P_0x7f03a4a2be20_1&reconnect=auto" class="pyvi…

None

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [14]:
from numpy.linalg import norm

def method2(mesh, detail="low"):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    if detail == "normal":
        target_len = diag_len * 5e-3
    elif detail == "high":
        target_len = diag_len * 2.5e-3
    elif detail == "low":
        target_len = diag_len * 2e-2
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.0, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [15]:
repaired_submesh = method2(submesh)

Target resolution: 0.06404882604239362 mm
#v: 607
#v: 606


In [16]:
visualize(repaired_submesh, filename="Repaired_submesh")

Widget(value='<iframe src="http://localhost:44475/index.html?ui=P_0x7f03a49171f0_2&reconnect=auto" class="pyvi…

None

#### Align submesh
Since we have modified the submesh, we need to change its boundary to what it used to be. Otherwise, when we combine the submesh and the remaining component later, there will be some small gaps.

In [17]:
aligned_submesh = align_submesh_boundary(remaining_mesh, repaired_submesh)

In [18]:
visualize(aligned_submesh, filename="Method2")

Widget(value='<iframe src="http://localhost:44475/index.html?ui=P_0x7f0376b8e320_3&reconnect=auto" class="pyvi…

None

#### Merge two meshes

In [19]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

#### Final refinement

In [20]:
final = pymesh.compute_outer_hull(repaired_full)
final, __ = pymesh.collapse_short_edges(final, 1e-5)
final = pymesh.resolve_self_intersection(final)

In [21]:
self_intersection_stats(final)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     994 |
+-----------------------------------+---------+
| Number of faces                   |    1986 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [22]:
visualize(final, filename="Final Outerhull")

Widget(value='<iframe src="http://localhost:44475/index.html?ui=P_0x7f036546d0f0_4&reconnect=auto" class="pyvi…

None

In [22]:
pymesh.save_mesh("final.ply", final, ascii=True)